<a href="https://colab.research.google.com/github/waynerun1010/Fluffy/blob/master/%E6%8A%93%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install pandas openpyxl

import pandas as pd
import requests
from google.colab import files
from bs4 import BeautifulSoup

url = 'https://www.cpbl.com.tw/player/trans'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 這將會在非 200 響應時拋出 HTTPError
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(soup)
except requests.exceptions.HTTPError as errh:
    print ("Http Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("OOps: Something Else",err)



table = soup.find('table')
if table:
    rows = table.find_all('tr')
else:
    print("No table found")

rows = table.find_all('tr')
headers = [header.text.strip() for header in rows[0].find_all('th')]

data = []
last_date = None  # 用來記錄最後讀到的日期
included_players = ['布藍登', '吳世豪']  # 要保留的特定球員名單

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]

    # 檢查當前行是否包含日期，假設日期格式為 'YYYY/MM/DD'
    if len(cols) > 0 and '/' in cols[0]:
        last_date = cols[0]  # 更新最後讀到的日期
    else:
        # 如果當前行沒有日期，則使用最後讀到的日期
        if last_date is not None:
            cols.insert(0, last_date)  # 在列表前插入最近的日期

    # 檢查是否包含 "富邦悍將" 這個關鍵詞
    if "富邦悍將" in cols and any(player in cols for player in included_players):
        data.append(cols)

df = pd.DataFrame(data, columns=headers)

print(df)


# 將DataFrame保存到Excel文件
df.to_excel("trans.xlsx", index=False, engine='openpyxl')

# 提供一個下載鏈接
files.download('trans.xlsx')


         異動日期   球員    球隊 異動原因
0  2024/04/21  布藍登  富邦悍將  升一軍
1  2024/04/19  吳世豪  富邦悍將  升一軍
2  2024/04/06  布藍登  富邦悍將  降二軍


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>